# Grokking LIME: How can we explain why an image classifier "knows" what’s in a photo without looking inside the model?

Kilian Kluge, [Inlinity](https://www.inlinity.ai) & [XAI Studio](https://www.xai-studio.de)

[https://github.com/ionicsolutions/grokking-lime](https://github.com/ionicsolutions/grokking-lime)


- GitHub: [ionicsolutions](https://github.com/ionicsolutions)
- Twitter: [@kilian_kluge](https://www.twitter.com/kilian_kluge)

<div style="width: 100%; margin-bottom: 100px;">
    <div><a href="https://www.inlinity.ai"><img src="assets/inlinity_logo_electric_blue.png" style="height:60px; margin-bottom: 30px;" /></a></div><div>
    <a href="https://www.xai-studio.de"><img src="assets/xai-studio-logo-m.png" style="height:60px;" /></a></div>
</div>

## Explainable AI? XAI? Interpretable Machine Learning?

<img src="assets/opaque_interpretable.png" width="500"/>

<img src="assets/explainable.png" width="500" />
<div style="height: 230px;" />

## NIST's Four Principles of XAI
![NIST's Four Principles of XAI](assets/four_principles.png)
<div style="height: 200px;" />

## Preparations

In [ ]:
# Essentials

import numpy as np
from PIL import Image

In [ ]:
# Visualization aids

import pandas as pd
from skimage.segmentation import mark_boundaries

def show_image(array: np.ndarray) -> Image:
    return Image.fromarray(np.uint8(array))

def show_segments(image: np.ndarray, segment_mask: np.ndarray) -> Image:
    return show_image(255 * mark_boundaries(image, segment_mask))

<div style="height: 200px;" />

## Load an image

In [ ]:
full_image = Image.open("camera.png")
full_image

In [ ]:
image = np.array(full_image)
image.shape

In [ ]:
image[15,77,0]

<div style="height: 200px;" />

## Load an image classification model

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import (
    MobileNetV2, preprocess_input, decode_predictions
)

In [ ]:
model = MobileNetV2()

In [ ]:
model_prediction = model.predict(preprocess_input(image[None,:,:,:]))
model_prediction.shape

In [ ]:
model_prediction[0,:10]

In [ ]:
np.argmax(model_prediction[0])

In [ ]:
decode_predictions(model_prediction)

<img src="assets/opaque.png" width="600" />

## LIME: Local Interpretable Model-Agnostic Explanations

Ribeiro et al. (2016): ["Why Should I Trust You?": Explaining the Predictions of Any Classifier](https://arxiv.org/abs/1602.04938)


1. [Segment the image](#Step-1:-Segment-the-image)
2. [Generate samples](#Step-2:-Generate-samples)
3. [Generate images](#Step-3:-Generate-images)
4. [Predict images](#Step-4:-Predict-images)
5. [Fit linear model](#Step-5:-Fit-linear-model)
6. [Generate visual explanation](#Step-6:-Generate-visual-explanation)

<div style="height: 200px;" />

## Step 1: Segment the image

In [ ]:
full_image

In [ ]:
from skimage.segmentation import felzenszwalb

In [ ]:
## live-code
segment_mask = felzenszwalb(image, scale=300)

In [ ]:
## live-code
show_segments(image, segment_mask)

In [ ]:
## live-code
segment_mask.shape

In [ ]:
## live-code
segment_mask

In [ ]:
print("Number of segments:", np.max(segment_mask) + 1)

<div style="height: 300px;" />

## Step 2: Generate samples

In [ ]:
def generate_samples(segment_mask: np.ndarray, num_of_samples: int) -> np.ndarray:
    ## live-code
    num_of_segments = np.max(segment_mask) + 1

    samples = np.random.rand(num_of_samples, num_of_segments) > 0.4
    
    ## /live-code
    return samples.astype(int)

In [ ]:
samples = generate_samples(segment_mask, 128)
samples.shape

In [ ]:
## live-code
pd.DataFrame(samples).head()

<div style="height: 300px;" />

## Step 3: Generate images

In [ ]:
def generate_images(image: np.ndarray,
                    segment_mask: np.ndarray,
                    samples: np.ndarray) -> np.ndarray:
    ## live-code
    images = []
    
    for sample in samples:
        
        mask = np.zeros(shape=segment_mask.shape, dtype=int)
        
        for segment_id in np.unique(segment_mask):
            
            if sample[segment_id] == 1:
                
                mask[segment_mask == segment_id] = 1
        
        images.append(mask[:, :, None] * image)
        
    
    return np.array(images)

In [ ]:
images = generate_images(image, segment_mask, samples)
images.shape

In [ ]:
## live-code
show_image(images[18])

<div style="height: 300px;" />

## Step 4: Predict images

In [ ]:
## live-code
predictions = model.predict(preprocess_input(images))

In [ ]:
predictions.shape

<div style="height: 300px;" />

## Step 5: Fit linear model

In [ ]:
from sklearn.linear_model import BayesianRidge
linear_model = BayesianRidge()

In [ ]:
pd.DataFrame(samples).head()

In [1]:
distances = np.linalg.norm(samples, axis=1)

NameError: name 'np' is not defined

In [ ]:
## live-code
linear_model.fit(samples, predictions[:, 759], sample_weight=distances)

In [ ]:
linear_model.coef_.shape

In [ ]:
linear_model.coef_[:10]

<div style="height: 300px;" />

## Step 6: Generate visual explanation

In [ ]:
from visualime.explain import render_explanation

In [ ]:
render_explanation(image,
                   segment_mask,
                   linear_model.coef_,
                   positive="green",
                   negative="violet")

<div style="height: 300px;" />

## Links & further reading

- This talk/notebook:
  - [github.com/ionicsolutions/grokking-lime](https://github.com/ionicsolutions/grokking-lime)
- Papers:
  - Phillips et al. (2021): [Four Principles of Explainable AI](https://nvlpubs.nist.gov/nistpubs/ir/2021/NIST.IR.8312.pdf) (NIST)
  - Ribeiro et al. (2016): ["Why Should I Trust You?": Explaining the Predictions of Any Classifier](https://arxiv.org/abs/1602.04938)
- LIME implementations:
  - [Original LIME on GitHub](https://github.com/marcotcr/lime) & [on PyPI](https://pypi.org/project/lime/)
  - [VisuaLIME on GitHub](https://github.com/xai-demonstrator/visualime) & [on PyPI](https://pypi.org/project/visualime/)